In [90]:
import ast, astor
import json

In [68]:
file_content = open('design.py', 'r').read()
ast_tree = ast.parse(file_content)

In [69]:
ast_tree

In [70]:
s = ast_tree.body[0].value.s

In [72]:
if s.startswith('@nni.architecture'):
    block = ast.parse(s[len('@nni.architecture')+1:])

In [67]:
{
    block_name: block_1,
    block_inputs: [x_t, h_t-1],
    block_output_aggregate: aggregate_free_outs,
    block_outputs: [h_t],

    layer_1: {
        layer_choice: [tanh, ReLU, identity, sigmoid],
        input_candidates: [images],
        input_num: 1,
        input_aggregate: null,
        outputs: [out],
    },

    layer_2: {
        layer_choice: [tanh, ReLU, identity, sigmoid],
        input_candidates: [layer_[1:1]/out],
        input_switch: choose_one,
        outputs: [out],
    },

    layer_3: {
        layer_choice: [tanh, ReLU, identity, sigmoid],
        input_candidates: [layer_[1:2]/out],
        input_switch: choose_one,
        outputs: [out],
    }
}

NameError: name 'block_name' is not defined

In [76]:
type(.keys[0].id)

str

In [78]:
block_dict = block.body[0].value
for key, value in zip(block_dict.keys, block_dict.values):
    if isinstance(value, ast.Dict):
        print(key.id)

layer_1
layer_2
layer_3


In [73]:
def foo(ast_tree):
    print('\n'.join(re.split('\n|\\\\n', astor.dump_tree(ast_tree))))

In [133]:
astor.to_source(block)

'{block_name: block_1, block_inputs: [x_t, h_t - 1], block_output_aggregate:\n    aggregate_free_outs, block_outputs: [h_t], layer_1: {layer_choice: [\n    tanh, ReLU, identity, sigmoid], input_candidates: [images], input_num: \n    1, input_aggregate: null, outputs: [out]}, layer_2: {layer_choice: [\n    tanh, ReLU, identity, sigmoid], input_candidates: [layer_[1:1] / out],\n    input_switch: choose_one, outputs: [out]}, layer_3: {layer_choice: [\n    tanh, ReLU, identity, sigmoid], input_candidates: [layer_[1:2] / out],\n    input_switch: choose_one, outputs: [out]}}\n'

In [134]:
block

In [136]:
import copy
bbb = copy.deepcopy(block)

In [177]:
class RewriteName(ast.NodeTransformer):

    def visit_Name(self, node):
        return ast.Str(s=node.id)

In [178]:
rewriteName = RewriteName()
aaa=rewriteName.visit(bbb)

In [181]:
print(astor.to_source(bbb))

{'block_name': 'block_1', 'block_inputs': ['x_t', 'h_t' - 1],
    'block_output_aggregate': 'aggregate_free_outs', 'block_outputs': [
    'h_t'], 'layer_1': {'layer_choice': ['tanh', 'ReLU', 'identity',
    'sigmoid'], 'input_candidates': ['images'], 'input_num': 1,
    'input_aggregate': 'null', 'outputs': ['out']}, 'layer_2': {
    'layer_choice': ['tanh', 'ReLU', 'identity', 'sigmoid'],
    'input_candidates': ['layer_'[1:1] / 'out'], 'input_switch':
    'choose_one', 'outputs': ['out']}, 'layer_3': {'layer_choice': ['tanh',
    'ReLU', 'identity', 'sigmoid'], 'input_candidates': ['layer_'[1:2] /
    'out'], 'input_switch': 'choose_one', 'outputs': ['out']}}



In [180]:
ast.dump(bbb)

"Module(body=[Expr(value=Dict(keys=[Str(s='block_name'), Str(s='block_inputs'), Str(s='block_output_aggregate'), Str(s='block_outputs'), Str(s='layer_1'), Str(s='layer_2'), Str(s='layer_3')], values=[Str(s='block_1'), List(elts=[Str(s='x_t'), BinOp(left=Str(s='h_t'), op=Sub(), right=Num(n=1))], ctx=Load()), Str(s='aggregate_free_outs'), List(elts=[Str(s='h_t')], ctx=Load()), Dict(keys=[Str(s='layer_choice'), Str(s='input_candidates'), Str(s='input_num'), Str(s='input_aggregate'), Str(s='outputs')], values=[List(elts=[Str(s='tanh'), Str(s='ReLU'), Str(s='identity'), Str(s='sigmoid')], ctx=Load()), List(elts=[Str(s='images')], ctx=Load()), Num(n=1), Str(s='null'), List(elts=[Str(s='out')], ctx=Load())]), Dict(keys=[Str(s='layer_choice'), Str(s='input_candidates'), Str(s='input_switch'), Str(s='outputs')], values=[List(elts=[Str(s='tanh'), Str(s='ReLU'), Str(s='identity'), Str(s='sigmoid')], ctx=Load()), List(elts=[BinOp(left=Subscript(value=Str(s='layer_'), slice=Slice(lower=Num(n=1), up

In [79]:
foo(ast_tree)

Module(
    body=[
        Expr(
            value=Str(
                s='@nni.architecture
{
    block_name: block_1,
    block_inputs: [x_t, h_t-1],
    block_output_aggregate: aggregate_free_outs,
    block_outputs: [h_t],

    layer_1: {
        layer_choice: [tanh, ReLU, identity, sigmoid],
        input_candidates: [images],
        input_num: 1,
        input_aggregate: null,
        outputs: [out],
    },

    layer_2: {
        layer_choice: [tanh, ReLU, identity, sigmoid],
        input_candidates: [layer_[1:1]/out],
        input_switch: choose_one,
        outputs: [out],
    },

    layer_3: {
        layer_choice: [tanh, ReLU, identity, sigmoid],
        input_candidates: [layer_[1:2]/out],
        input_switch: choose_one,
        outputs: [out],
    }
}
')),
        Assign(targets=[Name(id='layer_1_input_candidate')],
            value=ListComp(
                elt=Subscript(
                    value=Subscript(
                        value=Subscript(value=Name(id='gr

In [82]:
None

In [86]:
ast.dump(ast.parse('''layer_1_input_candidate = [graph['layer_1']['input_candidates'][x] for x in nni.get_candidate('layer_1')]
if graph['layer_1']['input_aggregate'] is not None:
    layer_1_aggregated_output = graph['layer_1']['input_aggregate'](*layer_1_input_candidate,)
layer_1_output = graph['layer_1']['layer_choice'][nni.get_layer_choice('layer_1')](layer_1_aggregated_output)'''))

"Module(body=[Assign(targets=[Name(id='layer_1_input_candidate', ctx=Store())], value=ListComp(elt=Subscript(value=Subscript(value=Subscript(value=Name(id='graph', ctx=Load()), slice=Index(value=Str(s='layer_1')), ctx=Load()), slice=Index(value=Str(s='input_candidates')), ctx=Load()), slice=Index(value=Name(id='x', ctx=Load())), ctx=Load()), generators=[comprehension(target=Name(id='x', ctx=Store()), iter=Call(func=Attribute(value=Name(id='nni', ctx=Load()), attr='get_candidate', ctx=Load()), args=[Str(s='layer_1')], keywords=[]), ifs=[], is_async=0)])), If(test=Compare(left=Subscript(value=Subscript(value=Name(id='graph', ctx=Load()), slice=Index(value=Str(s='layer_1')), ctx=Load()), slice=Index(value=Str(s='input_aggregate')), ctx=Load()), ops=[IsNot()], comparators=[NameConstant(value=None)]), body=[Assign(targets=[Name(id='layer_1_aggregated_output', ctx=Store())], value=Call(func=Subscript(value=Subscript(value=Name(id='graph', ctx=Load()), slice=Index(value=Str(s='layer_1')), ctx

In [98]:
node = ast.parse('''layer_1_input_candidate = [graph['layer_1']['input_candidates'][x] for x in nni.get_candidate('layer_1')]
if graph['layer_1']['input_aggregate'] is not None:
    layer_1_aggregated_output = graph['layer_1']['input_aggregate'](*layer_1_input_candidate,)
layer_1_output = graph['layer_1']['layer_choice'][nni.get_layer_choice('layer_1')](layer_1_aggregated_output)''')
print(astor.to_source(node))

layer_1_input_candidate = [graph['layer_1']['input_candidates'][x] for x in
    nni.get_candidate('layer_1')]
if graph['layer_1']['input_aggregate'] is not None:
    layer_1_aggregated_output = graph['layer_1']['input_aggregate'](*
        layer_1_input_candidate)
layer_1_output = graph['layer_1']['layer_choice'][nni.get_layer_choice(
    'layer_1')](layer_1_aggregated_output)



In [108]:
astor.to_source("ast.parse('"djfkslfk"')")

AttributeError: No defined handler for node of type str

In [99]:
ast.dump(ast.parse("foo(*layer_1_input_candidate,)"))

"Module(body=[Expr(value=Call(func=Name(id='foo', ctx=Load()), args=[Starred(value=Name(id='layer_1_input_candidate', ctx=Load()), ctx=Load())], keywords=[]))])"

In [100]:
ast.dump(ast.parse("foo(*layer_1_input_candidate)"))

"Module(body=[Expr(value=Call(func=Name(id='foo', ctx=Load()), args=[Starred(value=Name(id='layer_1_input_candidate', ctx=Load()), ctx=Load())], keywords=[]))])"

In [106]:
def expand_layer(layer_name):
    template = '''layer_name_input_candidate = [graph['layer_name']['input_candidates'][x] for x in nni.get_candidate('layer_name')]
if graph['layer_name']['input_aggregate'] is not None:
    layer_name_aggregated_output = graph['layer_name']['input_aggregate'](*layer_name_input_candidate,)
layer_name_output = graph['layer_name']['layer_choice'][nni.get_layer_choice('layer_name')](layer_name_aggregated_output)'''
    template = template.replace('layer_name', layer_name)
    return template

In [107]:
expand_layer('fuck')

"fuck_input_candidate = [graph['fuck']['input_candidates'][x] for x in nni.get_candidate('fuck')]\nif graph['fuck']['input_aggregate'] is not None:\n    fuck_aggregated_output = graph['fuck']['input_aggregate'](*fuck_input_candidate,)\nfuck_output = graph['fuck']['layer_choice'][nni.get_layer_choice('fuck')](fuck_aggregated_output)"

In [111]:
node=ast.parse("foo(*layer_1_input_candidate,)")

In [122]:
a = node.body[0].value.args[0]
#node.body[0].value.args = [ast.Tuple(a)]

In [127]:
see=ast.Tuple(elts=[a], ctx=ast.Store())

In [128]:
ast.dump(see)

"Tuple(elts=[Starred(value=Name(id='layer_1_input_candidate', ctx=Load()), ctx=Load())], ctx=Store())"

In [129]:
node.body[0].value.args[0] = see

In [130]:
astor.to_source(node)

'foo((*layer_1_input_candidate,))\n'

In [119]:
ast.dump(ast.parse("*a, = [1,2,3,4]"))

"Module(body=[Assign(targets=[Tuple(elts=[Starred(value=Name(id='a', ctx=Store()), ctx=Store())], ctx=Store())], value=List(elts=[Num(n=1), Num(n=2), Num(n=3), Num(n=4)], ctx=Load()))])"

In [120]:
astor.to_source(ast.parse("*a, = [1,2,3,4]"))

'*a, = [1, 2, 3, 4]\n'